## 웹 스크래핑(scraping) 기초: requests and BeautifulSoup
- https://velog.io/@yuns_u/python%EC%9C%BC%EB%A1%9C-web-scraping
- 여러 유형의 웹페이지를 스크래핑하여 파싱하는 방법
- 웹과의 통신을 도와주는 requests 라이브러리와 받은 웹 문서를 파싱(parsing)하는 BeautifulSoup 라이브러리를 이용한다.

#### 1. 웹페이지 요청하기
- "The Adventures of Sherlock Holmes" (by Arthur Conan Doyle) (https://ko.wikipedia.org/wiki/%EC%85%9C%EB%A1%9D_%ED%99%88%EC%A6%88%EC%9D%98_%EB%AA%A8%ED%97%98)

In [1]:
# !pip install requests
import requests

In [ ]:
url = 'https://www.gutenberg.org/cache/epub/1661/pg1661-images.html'
resp = requests.get(url)
print(resp.status_code)

#### 요청한 웹 페이지에서 필요 요소 추출하기 

In [2]:
# !pip install beautifulsoup4
from bs4 import BeautifulSoup

In [152]:
url = 'https://www.gutenberg.org/cache/epub/1661/pg1661-images.html'
page = requests.get(url)
page_content = page.content
# page_content = page.content.decode('utf-8')

parsed_page = BeautifulSoup(page_content, 'html.parser')

In [ ]:
print(parsed_page.contents)

#### 분석 웹페이지에서 정보 추출하기(1)
분석이 완료된 웹페이지 정보는 태그 이름을 이용하여 내용을 확인할 수 있다. 만약 __<title>...</title>__ 내용을 확인하고 싶으면 다음과 같이 분석 완료된 웹 데이터 다음에 __.태그명__ 을 붙이면 된다.

In [ ]:
print(parsed_page.title)

#### 분석 웹페이지에서 정보 추출하기(2)
그러나 위의 방법은 복잡한 웹 페이지의 구조에서 정보를 추출하기에 비효율적인 방법이다. **find()**, **find_all()**, **select()** 등의 함수를 이용하여 정보를 추출하는 것이 보다 효율적이다. 웹 브라우저에서 페이지 구조를 열어 원하는 정보의 태그를 확인하여 이 태그들만 추출하여 정보를 얻는 것이 효율적이다.

##### 제목과 서지 정보 등을 제외하고 소설의 본문만 가져오기
- 웹 페이지에서 가져오고 싶은 내용이 페이지의 소스에서 어떻게 표현되는지를 알려면 가져오고 싶은 부분을 일부 영역 선택한 후에 마우스 오른 클릭을 하고 Inspect 메뉴를 선택한다. 오른쪽 혹은 하단에서 나타나는 웹페이지의 소스 정보(Elements 정보)에서 해당 부분의 태그 위치가 반전되어 해당 부분의 태그를 찾을 수 있다.

- 소설의 본문은 모두 <p>...</p> 태그로 둘러싸여 있음을 알 수 있다. 이 <p>...</p> 사이의 내용을 모두 가져오면 소설의 본문을 텍스트 파일로 만들 수 있다. 

In [ ]:
paras = parsed_page.find_all(name='p')
print("All paragraph length is", len(paras), "\n")

for para in paras[:5]:
    print(para.get_text())

서지 정보를 제외하고 순수 본문만 추출하기 위해 추출 대상의 범위를 좁혀야 한다. 웹 페이지 소스를 잘 살펴보면 순수 본문 즉 *body*에 속하는 __<p></p>__ 태그는 __<div class='chapter'></div>__로 둘써싸여 있음을 알 수 있다. 따라서 각 chapter를 먼저 추출하고 이 chapter들에서 <p>를 추출하면 순수 본문만 추출할 수 있다.

In [ ]:
chapters = parsed_page.find_all(name='div', class_='chapter')
# chapters = parsed_page.find_all('div', 'chapter')
print('The number of chapters is ', len(chapters))

각 chapter에서 __<p>__ 즉 paragraph 정보 가져오기

In [ ]:
paragraphs = []

for chap in chapters:
    paras = chap.find_all(name='p')
    para_texts = []
    for para in paras:
        para_txt = para.get_text().strip()
        para_texts.append(para_txt)
    paragraphs.append(para_texts)

In [ ]:
print(len(paragraphs)); print(len(paragraphs[0])); print(len(paragraphs[1]))

In [ ]:
print(paragraphs[0][:10]); print(paragraphs[0][0])

##### 각 챕터의 제목만 추출하기

In [ ]:
chapter_titles = []

for chap in chapters:
    title = chap.find(name='h2')
    title = title.get_text()
    # title = chap.find(name='h2').get_text()
    chapter_titles.append(title)

print(chapter_titles)

#### 웹페이지에서 원하는 정보를 찾이오기 위한 함수들에는 **find()**, **find_all()**, **selector()**, **selector_one()** 등이 있다. 
##### 이 함수들의 차이를 또 다른 웹페이지(네이버 지식인 '허먼 멜빌' 검색 결과 첫 번째 페이지)를 통해 알아보자. 웹페이지 내용은 아래와 같다. 이 페이지에서 질문 제목, 질문 내용, 질문 날짜, 답변 갯수 등 4개 정보를 추출해 보자.

In [ ]:
import re

url = 'https://kin.naver.com/search/list.naver?query=%ED%97%88%EB%A8%BC+%EB%A9%9C%EB%B9%8C'
source = requests.get(url)
source = source.content

webpage = BeautifulSoup(source, 'html.parser')
q_list = webpage.find(name='ul', class_='basic1')
q_list = q_list.find_all(name='li')
# q_list = webpage.find(name='ul', class_='basic1').find_all(name='li')

print('The number of q_list is', len(q_list))

q_titles = []
q_texts = []
q_dates = []
replies = []
for q in q_list:
    tit = q.find(name='a').get_text()
    date = q.find(name='dd', class_='txt_inline').get_text()
    txt = q.find(name='dd', attrs={'class': None}).get_text()
    repl = q.find(name='span', class_='hit').get_text()
    repl = int(re.sub('답변수 ', '', repl))

    q_titles.append(tit)
    q_texts.append(txt)
    q_dates.append(date)
    replies.append(repl)

print('The number of q_titles is', len(q_list))
print('The number of q_texts is', len(q_list))
print('The number of q_dates is', len(q_list))
print('The number of replies is', len(q_list))

In [ ]:
print(q_titles[:3]); print(q_texts[:3]); print(q_dates[:3]); print(replies[:3])

##### IMDb 사이트의 Top 250 Movies 페이지에서 영화 이미지 url, 영화 제목, 영화 소개 url, 순위, 상영 연도, 평점 등의 정보를 추출해 보기
(https://www.imdb.com/chart/top?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=470df400-70d9-4f35-bb05-8646a1195842&pf_rd_r=74D09HPMTCCHQ2GNQSST&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_ql_3)


In [29]:
import re

url = 'https://www.imdb.com/chart/top?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=470df400-70d9-4f35-bb05-8646a1195842&pf_rd_r=74D09HPMTCCHQ2GNQSST&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_ql_3'
source = requests.get(url)
source = source.content
webpage = BeautifulSoup(source, 'html.parser')

movie_list = webpage.find(name='tbody').find_all(name='tr')

poster_image_urls = []
movie_titles = []
movie_urls = []
movie_ranks = []
movie_years = []
movie_ratings = []
for movie in movie_list:
    poster_tag = movie.find(name='td').find('img')
    poster_url = poster_tag.attrs['src']
    poster_image_urls.append(poster_url)
    
    title_col = movie.find(name='td', class_='titleColumn')
    title = title_col.find('a').get_text().strip()
    title_url = title_col.find('a').attrs['href']
    movie_titles.append(title)
    movie_urls.append(title_url)

    x = title_col.get_text().strip()
    rank = re.search('^\d{1,3}', x).group()
    year = re.search('\d{4}', x).group()    
    # rank = re.search('^\d{1,3}', title_col.get_text().strip()).group()
    # year = re.search('\d{4}', title_col.get_text().strip()).group()
    movie_ranks.append(rank)
    movie_years.append(year)
    
    rating = movie.find(name='td', class_='ratingColumn imdbRating').get_text().strip()
    movie_ratings.append(rating)

In [ ]:
print(poster_image_urls[:3]); print(movie_titles[:3]); print(movie_urls[:3]); print(movie_ranks[:3]); print(movie_years[:3]); print(movie_ratings[:3])

- 이미지 url을 이용하여 이미지를 다운로드할 때는 다음과 같이 실행한다.

In [ ]:
img_url = poster_image_urls[0]
response = requests.get(img_url)
with open('../images/poster_image_01.jpg', 'wb') as img:
    img.write(response.content)